In [11]:
from model import model_manual
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
# To create Test , Train and Validation DataGenerator

def preprocess_data(train_dir, val_dir, test_dir, target_size=(128, 512), batch_size=32):
    train_datagen = ImageDataGenerator()
    
    val_datagen = ImageDataGenerator()
    test_datagen = ImageDataGenerator()
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    return train_generator, validation_generator, test_generator

In [3]:
train_dir = './dataset/train'
val_dir = './dataset/validation'
test_dir = './dataset/test'
train_gen, val_gen, test_gen = preprocess_data(train_dir, val_dir, test_dir)

Found 36605 images belonging to 21 classes.
Found 3150 images belonging to 21 classes.
Found 3150 images belonging to 21 classes.


In [13]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [ ]:
with strategy.scope():

    model = model_manual(21)
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor = 'val_accuracy',
                              patience = 5, mode = 'max',
                              restore_best_weights = True)

    checkpoint = ModelCheckpoint('best_model.keras',
                            monitor = 'val_accuracy',
                            save_best_only = True)
    history = model.fit(
        train_gen,
        epochs=10,  # Adjust the number of epochs
        validation_data=val_gen
    )

Epoch 1/10


C:\Users\DC\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 838/1144 ━━━━━━━━━━━━━━━━━━━━ 13:07 3s/step - accuracy: 0.2589 - loss: 5.9738

In [ ]:
model.save("manual_cnn.keras")